In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=d5040f4dab2bb51b60ee3b91eda75b7d07e25eb027d4055937148f40e04c8197
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [4]:
spark = (SparkSession.builder.master("local[*]")
                    .config("spark.driver.memory","16g")
                    .config("spark.sql.shuffle.partitions",200)
                    .appName('Amex_Default')
                    .master('local')
                    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 04:59:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
train_path = "../input/amex-default-prediction/train_data.csv"
test_path = "../input/amex-default-prediction/test_data.csv"
label_path = "../input/amex-default-prediction/train_labels.csv"

In [6]:
train_df = spark.read.options(header='True',  delimiter=',').csv(train_path)
test_df = spark.read.options(header='True', delimiter=',').csv(test_path)
label_df = spark.read.options(header='True', delimiter=',').csv(label_path)

In [7]:
train_df.printSchema()

root
 |-- customer_ID: string (nullable = true)
 |-- S_2: string (nullable = true)
 |-- P_2: string (nullable = true)
 |-- D_39: string (nullable = true)
 |-- B_1: string (nullable = true)
 |-- B_2: string (nullable = true)
 |-- R_1: string (nullable = true)
 |-- S_3: string (nullable = true)
 |-- D_41: string (nullable = true)
 |-- B_3: string (nullable = true)
 |-- D_42: string (nullable = true)
 |-- D_43: string (nullable = true)
 |-- D_44: string (nullable = true)
 |-- B_4: string (nullable = true)
 |-- D_45: string (nullable = true)
 |-- B_5: string (nullable = true)
 |-- R_2: string (nullable = true)
 |-- D_46: string (nullable = true)
 |-- D_47: string (nullable = true)
 |-- D_48: string (nullable = true)
 |-- D_49: string (nullable = true)
 |-- B_6: string (nullable = true)
 |-- B_7: string (nullable = true)
 |-- B_8: string (nullable = true)
 |-- D_50: string (nullable = true)
 |-- D_51: string (nullable = true)
 |-- B_9: string (nullable = true)
 |-- R_3: string (nullable = t

In [8]:
print(train_df.count())

5531451


In [9]:
print(len(train_df.columns))

190


In [10]:
train_df.select('customer_ID').show()

+--------------------+
|         customer_ID|
+--------------------+
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|0000099d6bd597052...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
|00000fd6641609c6e...|
+--------------------+
only showing top 20 rows



In [11]:
train_df.show(1, vertical=True)

23/01/02 05:03:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0---------------------------
 customer_ID | 0000099d6bd597052... 
 S_2         | 2017-03-09           
 P_2         | 0.9384687191272548   
 D_39        | 0.0017333390041739   
 B_1         | 0.0087244509498605   
 B_2         | 1.0068382339663076   
 R_1         | 0.0092277222786689   
 S_3         | 0.1240351558143743   
 D_41        | 0.0087711319938824   
 B_3         | 0.0047092406313857   
 D_42        | null                 
 D_43        | null                 
 D_44        | 0.000630134804911503 
 B_4         | 0.0809863324662527   
 D_45        | 0.708906305121159    
 B_5         | 0.1706002293387026   
 R_2         | 0.0062040314303209   
 D_46        | 0.3585865793715965   
 D_47        | 0.525351040810055    
 D_48        | 0.255736073902975    
 D_49        | null                 
 B_6  

In [12]:
label_df.show(1, vertical=True)

-RECORD 0---------------------------
 customer_ID | 0000099d6bd597052... 
 target      | 0                    
only showing top 1 row



In [13]:
train_df.groupby('customer_ID').count().show()

+--------------------+-----+
|         customer_ID|count|
+--------------------+-----+
|0012e41fe6caa3ba3...|   10|
|00582971eb87cf0bb...|   13|
|00dbda372d97f2357...|   13|
|00dc3b4e9f8f3b114...|   13|
|0119d38628b2cbdd0...|   13|
|0125e11ed5c94d63c...|   13|
|01439ee3abf1b4552...|   13|
|014acecc038c204f2...|    4|
|01500e2a9f82cfab4...|   13|
|0152860993fe1a7b1...|   13|
|0154c4e7b3d035cff...|   13|
|017c62c2d536df824...|   10|
|0181b485c84ad2b52...|   13|
|01bc5e75de384a9bc...|   13|
|01f3396eacb7b77ba...|   13|
|00093b69756b1afe3...|   13|
|003491636f5638e54...|   13|
|00d769dbf0a5e3e73...|   13|
|01014a6e1bba3352e...|   13|
|011716311fb15d39c...|   13|
+--------------------+-----+
only showing top 20 rows



In [14]:
train_df.tail(1)

[Row(customer_ID='fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681', S_2='2018-03-14', P_2='0.9821748500563192', D_39='0.0024737143489931', B_1='7.693872947067027e-05', B_2='0.9928799540835922', R_1='0.0008092971779174774', S_3='0.1191649070736343', D_41='0.0032872852275932', B_3='0.0140915687516334', D_42=None, D_43='0.0134545274860418', D_44='0.0067939187376779', B_4='0.0183351337196274', D_45='0.7507839475981496', B_5='0.0066170642528045', R_2='0.0010160315706084', D_46='0.4597452584302709', D_47='0.531609965358133', D_48='0.0185772188058246', D_49=None, B_6='0.2331172373259007', B_7='0.0281845334749846', B_8='0.0079197841038872', D_50='0.0886079774332406', D_51='0.0080573884318738', B_9='0.0019762605057766', R_3='0.0050144551043305', D_52='0.2812290068419038', P_3='0.6119449599433455', B_10='0.2913446438125343', D_53=None, S_5='0.0045800485943607', B_11='0.0056537890007912', S_6='0.0056094287068511', D_54='1.0035494118469506', R_4='0.0007625988621855695', S_7='0.065

In [15]:
label_df.groupby('target').count().show()

+------+------+
|target| count|
+------+------+
|     0|340085|
|     1|118828|
+------+------+



In [16]:
train_df.head(1)

[Row(customer_ID='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a', S_2='2017-03-09', P_2='0.9384687191272548', D_39='0.0017333390041739', B_1='0.0087244509498605', B_2='1.0068382339663076', R_1='0.0092277222786689', S_3='0.1240351558143743', D_41='0.0087711319938824', B_3='0.0047092406313857', D_42=None, D_43=None, D_44='0.000630134804911503', B_4='0.0809863324662527', D_45='0.708906305121159', B_5='0.1706002293387026', R_2='0.0062040314303209', D_46='0.3585865793715965', D_47='0.525351040810055', D_48='0.255736073902975', D_49=None, B_6='0.0639022133803909', B_7='0.0594157330614109', B_8='0.006465576798311', D_50='0.1486979558199995', D_51='1.3358557940752642', B_9='0.0082067391252636', R_3='0.0014225021561254', D_52='0.2073338786110817', P_3='0.7364627260945562', B_10='0.0962188066642168', D_53=None, S_5='0.0233811220509184', B_11='0.0027680616648439', S_6='0.0083216461012018', D_54='1.0015189798143298', R_4='0.0082984353659013', S_7='0.1613448213392926', B_12='0.14

In [17]:
from pyspark.sql.functions import countDistinct
plot_df=train_df.groupBy('S_2').agg(countDistinct('customer_ID')).show()

+----------+------------------+
|       S_2|count(customer_ID)|
+----------+------------------+
|2017-12-05|              9982|
|2017-05-14|              8073|
|2017-05-11|             12655|
|2017-12-22|             14695|
|2017-10-20|             17827|
|2018-03-16|             16395|
|2017-08-18|             14627|
|2017-04-30|              9372|
|2017-09-12|             14827|
|2018-03-12|             15916|
|2017-09-26|             14787|
|2017-05-18|             13603|
|2017-03-08|             13427|
|2018-01-19|             15956|
|2018-02-27|             18317|
|2017-03-18|             18702|
|2017-10-31|              9367|
|2018-02-23|             16770|
|2017-10-01|             10347|
|2017-10-12|             14623|
+----------+------------------+
only showing top 20 rows



In [18]:
df = train_df.join(label_df, on='customer_ID',how='left')

In [19]:
df.tail(1)

[Row(customer_ID='ffdbfc624676e06b1b8c5b45c8ce5fd08e1aad96797b6dd04635dc6cd91d2b0b', S_2='2018-03-16', P_2='0.743797666788825', D_39='0.2123381265884293', B_1='0.0419089638299679', B_2='0.0095866243635731', R_1='0.0048279331176906', S_3='0.1083136934309505', D_41='0.0015338771906982', B_3='0.0351484778761323', D_42=None, D_43=None, D_44='0.0021818561536708', B_4='0.1225068711103822', D_45='0.3349585183486029', B_5='0.0198218905198147', R_2='0.0006003109163680031', D_46='0.4368980627457388', D_47='0.285187784438158', D_48='0.1444423101571554', D_49=None, B_6='0.0324974553203276', B_7='0.1777244915200438', B_8='0.0087818714748739', D_50=None, D_51='0.3375769143583987', B_9='0.0238344490760452', R_3='0.0054820029131505', D_52='0.1042329426376955', P_3='0.6288065423737685', B_10='0.0209910013414105', D_53=None, S_5='0.0395282428615153', B_11='0.019131981513097', S_6='0.0033296948253406', D_54='1.0079460031774763', R_4='0.0016728695303207', S_7='0.0653531967068053', B_12='0.0243481634591622

In [20]:
df.groupby('target').count().show()

+------+-------+
|target|  count|
+------+-------+
|     0|4153582|
|     1|1377869|
+------+-------+



In [21]:
import pyspark.pandas as ps
from pyspark import StorageLevel
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [22]:
train_df.select("customer_ID").explain() 

== Physical Plan ==
FileScan csv [customer_ID#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/kaggle/input/amex-default-prediction/train_data.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<customer_ID:string>




In [23]:
def get_null_count(sql_df, colname):
    count = (sql_df.select(colname)
                   .filter(F.col(colname).isNull())
                   .count())
    return count

In [24]:
missing_customer_train = get_null_count(train_df, "customer_ID") 
missing_customer_test = get_null_count(test_df, "customer_ID")

total_miss = missing_customer_train + missing_customer_test
print(f"Missing customer_ID: {total_miss}")

Missing customer_ID: 0


In [25]:
def add_suffix(names, suffix):
    return [name + suffix for name in names]

In [26]:
info_cols = ['customer_ID', 'S_2']
target_cols = ['target']
cat_cols = [
    'B_30', 'B_38', 
    'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


# Define Numeric Columns
excluded = info_cols + cat_cols
num_cols = [col for col in train_df.columns if col not in excluded]

# Define Feature Columns
features_cols =  cat_cols + num_cols

print(f"Number of categoric cols: {len(cat_cols)}")
print(f"Number of numeric cols: {len(num_cols)}")

Number of categoric cols: 11
Number of numeric cols: 177


In [27]:
print(target_cols)

['target']


In [28]:
train_df = (train_df.fillna(0, subset=num_cols)
                    .fillna("null", subset=cat_cols))

test_df = (test_df.fillna(0, subset=num_cols)
                  .fillna("null", subset=cat_cols))

In [29]:
# Create columns aliases
cat_index_cols = add_suffix(cat_cols, "_index")

# Fit StringIndexer
indexers = StringIndexer(inputCols=cat_cols, outputCols=cat_index_cols)
indexers_model = indexers.fit(train_df)

# Transform to data
train_df_indexed = indexers_model.transform(train_df)
test_df_indexed = indexers_model.transform(test_df)

In [30]:
# Create columns aliases
cat_index_cols = add_suffix(cat_cols, "_index")

# Fit StringIndexer
indexers = StringIndexer(inputCols=cat_cols, outputCols=cat_index_cols)
indexers_model = indexers.fit(train_df)

# Transform to data
train_df_indexed = indexers_model.transform(train_df)
test_df_indexed = indexers_model.transform(test_df)

In [31]:
# See what columns the indexer handle
indexers.getInputCols()

# See the indexed columns
train_df_indexed.select("B_30_index").show(5)

+----------+
|B_30_index|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 5 rows



In [32]:
# Create columns aliases
cat_ohe_cols = add_suffix(cat_cols, "_ohe")

# Fit OneHotEncoder
ohe = OneHotEncoder(inputCols=cat_index_cols, outputCols=cat_ohe_cols)
ohe_model = ohe.fit(train_df_indexed)

# Transform to data
train_df_ohed = ohe_model.transform(train_df_indexed)
test_df_ohed = ohe_model.transform(test_df_indexed)

In [33]:
train_df_ohed.select("B_30_ohe").show(5)

+-------------+
|     B_30_ohe|
+-------------+
|(3,[0],[1.0])|
|(3,[0],[1.0])|
|(3,[0],[1.0])|
|(3,[0],[1.0])|
|(3,[0],[1.0])|
+-------------+
only showing top 5 rows



In [34]:
# Functions for each type
# each tuple consist of: (function, column's suffix)
num_funcs = [
    (F.mean, "_mean"),
    (F.stddev, "_std"),
    (F.min, "_min"),
    (F.max, "_max"),
]

cat_funcs = [
    (F.count, "_count"),
    (F.last, "_last"),
    (F.countDistinct, "_nunique"),
]

In [35]:
# Arguments for .agg method
# each arg consist of: func(colname).alias(colname + suffix)
import itertools
agg_num_args = [
    func(col).alias(col + suffix) 
    for col, (func, suffix) in itertools.product(num_cols, num_funcs)]

agg_cols_args = [
    func(col).alias(col + suffix) 
    for col, (func, suffix) in itertools.product(cat_ohe_cols, cat_funcs)]

# Combine numeric and categoric agg arguments
agg_args = agg_num_args + agg_cols_args
agg_args[0]

Column<'avg(P_2) AS P_2_mean'>

In [36]:
# Columns that we won't use
unused_cols = cat_cols + num_cols + cat_index_cols + cat_ohe_cols
print(f"Unused columns {len(unused_cols)}")

Unused columns 210


In [37]:
# Apply the agg while also dropping unused columns
train_df_grouped = (train_df_ohed.groupBy("customer_ID")
                                 .agg(*agg_cols_args)
                                 .drop(*unused_cols))

test_df_grouped = (test_df_ohed.groupBy("customer_ID")
                                .agg(*agg_cols_args)
                                .drop(*unused_cols))

In [38]:
train_joined_df = train_df_grouped.join(F.broadcast(label_df), on="customer_ID")

In [39]:
train_joined_df.head(1)

[Row(customer_ID='000084e5023181993c2e1b665ac88dbb1ce9ef621ec5370150fc2f8bdca6202c', B_30_ohe_count=13, B_30_ohe_last=SparseVector(3, {0: 1.0}), B_30_ohe_nunique=1, B_38_ohe_count=13, B_38_ohe_last=SparseVector(7, {0: 1.0}), B_38_ohe_nunique=2, D_114_ohe_count=13, D_114_ohe_last=SparseVector(2, {0: 1.0}), D_114_ohe_nunique=1, D_116_ohe_count=13, D_116_ohe_last=SparseVector(2, {0: 1.0}), D_116_ohe_nunique=1, D_117_ohe_count=13, D_117_ohe_last=SparseVector(7, {}), D_117_ohe_nunique=2, D_120_ohe_count=13, D_120_ohe_last=SparseVector(2, {0: 1.0}), D_120_ohe_nunique=1, D_126_ohe_count=13, D_126_ohe_last=SparseVector(3, {0: 1.0}), D_126_ohe_nunique=1, D_63_ohe_count=13, D_63_ohe_last=SparseVector(5, {0: 1.0}), D_63_ohe_nunique=1, D_64_ohe_count=13, D_64_ohe_last=SparseVector(4, {2: 1.0}), D_64_ohe_nunique=1, D_66_ohe_count=13, D_66_ohe_last=SparseVector(2, {0: 1.0}), D_66_ohe_nunique=1, D_68_ohe_count=13, D_68_ohe_last=SparseVector(7, {0: 1.0}), D_68_ohe_nunique=1, target='0')]

In [40]:
dim = len(train_joined_df.columns)
print(f"Total features: {dim}")

Total features: 35


In [41]:
va = VectorAssembler(
    inputCols=train_joined_df.drop("customer_ID", "target").columns,
    outputCol="features",
    handleInvalid="error",
)

train_ready_df = (va.transform(train_joined_df)
                    .select(["customer_ID", "features", "target"])
                    .persist(StorageLevel.DISK_ONLY))

test_ready_df = (va.transform(test_df_grouped)
                   .select(["customer_ID", "features"])
                   .persist(StorageLevel.DISK_ONLY))

In [42]:
train_ready_df.head(1)

[Row(customer_ID='0012e41fe6caa3ba31b55b3de2030cbb77b01203aeb4a5c6677de5b80f15cebe', features=SparseVector(66, {0: 10.0, 1: 1.0, 4: 1.0, 5: 10.0, 8: 1.0, 13: 1.0, 14: 10.0, 15: 1.0, 17: 2.0, 18: 10.0, 19: 1.0, 21: 2.0, 22: 10.0, 26: 1.0, 30: 2.0, 31: 10.0, 33: 1.0, 34: 2.0, 35: 10.0, 37: 1.0, 39: 2.0, 40: 10.0, 41: 1.0, 46: 1.0, 47: 10.0, 48: 1.0, 52: 2.0, 53: 10.0, 54: 1.0, 56: 1.0, 57: 10.0, 61: 1.0, 65: 2.0}), target='0')]

In [43]:
print(train_ready_df.schema['target'].dataType)

StringType()


In [44]:
from pyspark.sql.types import IntegerType
train_ready_df = train_ready_df.withColumn("target", train_ready_df["target"].cast(IntegerType()))

In [45]:
logres = LogisticRegression(featuresCol="features", labelCol="target")
logres_model = logres.fit(train_ready_df)

In [46]:
test_predictions = logres_model.transform(test_ready_df)
test_predictions

DataFrame[customer_ID: string, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [47]:
submission = (test_predictions.select(["customer_ID", "probability"])
                              .toPandas())

In [48]:
# Get probability of target 1
submission["probability"] = submission["probability"].apply(lambda x: x[-1])

# Rename columns
submission = submission.rename(columns={"probability": "prediction"})

# Save File
submission.to_csv("submission.csv", index=False)

In [49]:
pd.read_csv("submission.csv").head()

,customer_ID,prediction
0,00004ffe6e01e1b688170bbd108da8351bc4c316eacfef...,0.013636
1,00048ea2f1b75bb675a58fa26df939a0755f0331e04836...,0.549494
2,001d908c73a0ee3725921f865638747d28c5d338665675...,0.077047
3,0022f9fde71c6d3bc2a16f648c56a07cc0547c80a1428d...,0.508021
4,00323fa8e0aca38d4a75007282473d206ab8f5ac164c84...,0.735199
